<a href="https://colab.research.google.com/github/Tomawock/MLDM_COVID-19/blob/master/testing/covid_dtr_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import joblib
from datetime import datetime

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.metrics import r2_score, mean_squared_error, max_error
from sklearn.ensemble import RandomForestRegressor 

from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

In [20]:
path_global='/content/'
datasets_names= ['dataset_finale_medie_rnd1.csv','dataset_finale_medie_2_week_rnd1.csv','dataset_finale_medie_ponderate_rnd1.csv',
               'dataset_finale_medie_rnd220.csv','dataset_finale_medie_2_week_rnd220.csv','dataset_finale_medie_ponderate_rnd220.csv']

rdn_states= [1, 22, 777, 6654, 432145]
features_prediction= ['fut1_nuovi_positivi','fut2_nuovi_positivi',
                     'fut1_ricoverati_con_sintomi','fut2_ricoverati_con_sintomi',
                     'fut1_terapia_intensiva','fut2_terapia_intensiva',
                     'fut1_deceduti','fut2_deceduti',
                     'fut1_totale_ospedalizzati','fut2_totale_ospedalizzati',
                     'fut1_dimessi_guariti','fut2_dimessi_guariti']

coefficents=['R2','RMSE','MAXERR']

In [21]:
col=[]
inde=[]
for feature in features_prediction:
  for coeff in coefficents:
    col.append(coeff+'_'+feature)
    #print(f"Columns_names\t{coeff+'_'+feature}")

for rnd in rdn_states:
  for path in datasets_names:
    inde.append(path+':'+str(rnd))
    #print(f"indexes_\t{path+':'+str(rnd)}")

df_results=pd.DataFrame(columns=col,index=inde)
print(df_results.columns)
print(df_results.index)

Index(['R2_fut1_nuovi_positivi', 'RMSE_fut1_nuovi_positivi',
       'MAXERR_fut1_nuovi_positivi', 'R2_fut2_nuovi_positivi',
       'RMSE_fut2_nuovi_positivi', 'MAXERR_fut2_nuovi_positivi',
       'R2_fut1_ricoverati_con_sintomi', 'RMSE_fut1_ricoverati_con_sintomi',
       'MAXERR_fut1_ricoverati_con_sintomi', 'R2_fut2_ricoverati_con_sintomi',
       'RMSE_fut2_ricoverati_con_sintomi',
       'MAXERR_fut2_ricoverati_con_sintomi', 'R2_fut1_terapia_intensiva',
       'RMSE_fut1_terapia_intensiva', 'MAXERR_fut1_terapia_intensiva',
       'R2_fut2_terapia_intensiva', 'RMSE_fut2_terapia_intensiva',
       'MAXERR_fut2_terapia_intensiva', 'R2_fut1_deceduti',
       'RMSE_fut1_deceduti', 'MAXERR_fut1_deceduti', 'R2_fut2_deceduti',
       'RMSE_fut2_deceduti', 'MAXERR_fut2_deceduti',
       'R2_fut1_totale_ospedalizzati', 'RMSE_fut1_totale_ospedalizzati',
       'MAXERR_fut1_totale_ospedalizzati', 'R2_fut2_totale_ospedalizzati',
       'RMSE_fut2_totale_ospedalizzati', 'MAXERR_fut2_totale_osped

In [22]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start of all the code =", current_time)

Start of all the code = 09:56:29


In [23]:
# datasets_names= ['dataset_finale_medie_rnd1.csv','dataset_finale_medie_2_week_rnd1.csv']
# rdn_states= [1,22]
# features_prediction= ['fut1_nuovi_positivi','fut2_nuovi_positivi']

In [24]:
for rnd in rdn_states:
  for dataset in datasets_names:
    for feature in features_prediction:

      df = pd.read_csv(path_global+''+dataset, parse_dates=["Data"], infer_datetime_format=True)
      rnd_state = rnd
      predictor_columns = [
            'pass_Ammoniaca', 
            'pass_Benzene',
            'pass_Biossido di Azoto', 
            'pass_Biossido di Zolfo',
            'pass_Monossido di Azoto', 
            'pass_Monossido di Carbonio',
            'pass_Ossidi di Azoto', 
            'pass_Ozono', 
            'pass_PM10 (SM2005)',
            'pass_Particelle sospese PM2.5',
            'pass_Radiazione Globale', 
            'pass_Temperatura',
            'pass_deceduti',
            'pass_nuovi_positivi',
            'pass_ricoverati_con_sintomi', 
            'pass_tamponi']
      ################################################################################
      train = df.sample(frac=0.7, random_state=rnd_state)
      test = df.drop(train.index)

      train_X = train[predictor_columns]
      train_y = train[feature]

      test_X = test[predictor_columns]
      test_y = test[feature]
      ################################################################################
      dt_regressor = DecisionTreeRegressor(random_state=rnd_state)
      ################################################################################
      pipeline = Pipeline([("imp", SimpleImputer(missing_values=np.nan, strategy="mean")), 
                     ("dtr", dt_regressor)])
      ################################################################################
      criterion_list = ["mse", "mae", "friedman_mse"]
      splitter_list = ["best", "random"]
      max_depth_list = [None, 5, 6]
      min_samples_split_list = [2, 3]
      min_samples_split_leaf_list = [1, 2]
      min_weight_fraction_leaf_list = [0.0, 0.1]
      max_features_list = [None, "sqrt", "auto"]
      max_leaf_nodes_list = [None, 10, 25]
      min_impurity_decrease_list = [0.0, 0.1]

      param_grid = {"dtr__criterion": criterion_list,
                    "dtr__splitter": splitter_list,
                    "dtr__max_depth": max_depth_list,
                    "dtr__min_samples_split": min_samples_split_list,
                    "dtr__min_samples_leaf": min_samples_split_leaf_list,
                    "dtr__min_weight_fraction_leaf": min_weight_fraction_leaf_list,
                    "dtr__max_features": max_features_list,
                    "dtr__max_leaf_nodes": max_leaf_nodes_list,
                    "dtr__min_impurity_decrease": min_impurity_decrease_list}
      ################################################################################
      kfold_splits = 5
      grid_regressor = GridSearchCV(estimator=pipeline,  
                              n_jobs=-1, 
                              verbose=0,
                              return_train_score=True,
                              cv=KFold(n_splits=kfold_splits, shuffle=True),
                              param_grid=param_grid)

      grid_regressor.fit(train_X, train_y)
      ################################################################################
      best_regressor = grid_regressor.best_estimator_['dtr']

      imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
      imputer = imputer.fit(test_X)
      test_X_imp = imputer.transform(test_X)

      predicted = best_regressor.predict(test_X_imp)
      ################################################################################
      r2=float("{:.3f}".format(r2_score(test_y, predicted)))
      rmse=float("{:.3f}".format(mean_squared_error(test_y, predicted, squared=False)))
      maxerr=float("{:.3f}".format(max_error(test_y, predicted)))
      print(f"R2: \t\t{r2}")
      print(f"RMSE: \t\t{rmse}")
      print(f"MAX ERR: \t{maxerr}")
      ################################################################################
      df_results.loc[[dataset+':'+str(rnd_state)], [coefficents[0]+'_'+feature]] = r2
      df_results.loc[[dataset+':'+str(rnd_state)], [coefficents[1]+'_'+feature]] = rmse
      df_results.loc[[dataset+':'+str(rnd_state)], [coefficents[2]+'_'+feature]] = maxerr
      now = datetime.now()
      current_time = now.strftime("%H:%M:%S")
      print("END of one combination Cycle =", current_time)

KeyboardInterrupt: ignored

In [ ]:
df_results.to_csv('covid_random_forest_test.csv')